***we import libraries from python*** 

In [ ]:
import csv
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer

***here the dataset used for training and testing is read.To use your own datset just change the path***


In [ ]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/email_classifier/spam copy (1).csv')

In [ ]:
df.shape


(5572, 2)

In [ ]:
df

,Label,EmailText
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will ÃƒÂŒ_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


***In this cell any duplicates that are in the dataset are removed***


In [ ]:
df.drop_duplicates(inplace= True)

In [ ]:
df.shape

(5169, 2)

***this cell is to check if there are any NAN values***


In [ ]:
df.isnull().sum()

Label        0
EmailText    0
dtype: int64

***IN THIS CELL WE DEFINE HYPERMETERS AND CONVERT TEXT INTO SEQUENCED NUMBERS USING TOKONIZERS***

In [ ]:
vocab_size = 100000
embedding_dim = 16
max_length = 64
trunc_type='post'
oov= "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov)
tokenizer.fit_on_texts(df['EmailText'])
word_count = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['EmailText'])
pad = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

In [ ]:
word_count

{'<OOV>': 1,
 'i': 2,
 'to': 3,
 'you': 4,
 'a': 5,
 'the': 6,
 'u': 7,
 'and': 8,
 'in': 9,
 'is': 10,
 'me': 11,
 'my': 12,
 'for': 13,
 'your': 14,
 'it': 15,
 'of': 16,
 'have': 17,
 'call': 18,
 'that': 19,
 'on': 20,
 '2': 21,
 'are': 22,
 'now': 23,
 'so': 24,
 'but': 25,
 'not': 26,
 'can': 27,
 'at': 28,
 'or': 29,
 "i'm": 30,
 'do': 31,
 'be': 32,
 'get': 33,
 'will': 34,
 'just': 35,
 'if': 36,
 'with': 37,
 'we': 38,
 'no': 39,
 'this': 40,
 'ur': 41,
 'up': 42,
 'gt': 43,
 'lt': 44,
 '4': 45,
 'how': 46,
 'when': 47,
 'go': 48,
 'from': 49,
 'ok': 50,
 'out': 51,
 'all': 52,
 'what': 53,
 'free': 54,
 'know': 55,
 'like': 56,
 'then': 57,
 'got': 58,
 'good': 59,
 'come': 60,
 'am': 61,
 'was': 62,
 'time': 63,
 'its': 64,
 'only': 65,
 'day': 66,
 'want': 67,
 'love': 68,
 'there': 69,
 'he': 70,
 'text': 71,
 'send': 72,
 'going': 73,
 'one': 74,
 'need': 75,
 'by': 76,
 'lor': 77,
 'home': 78,
 'as': 79,
 'about': 80,
 'still': 81,
 'see': 82,
 'txt': 83,
 'back': 84,
 

In [ ]:
pad

array([[   0,    0,    0, ...,   58, 4014,  138],
       [   0,    0,    0, ...,  436,    7, 1769],
       [   0,    0,    0, ...,  708,  438, 4015],
       ...,
       [   0,    0,    0, ...,  100,  241, 8917],
       [   0,    0,    0, ...,  197,   13,   54],
       [   0,    0,    0, ...,    3,   64,  284]], dtype=int32)

***This cell splits the dats into training and testing dats by the ratio of 8 : 2***

In [ ]:
X_train,X_val,Y_train,Y_val=train_test_split(pad,df['Label'],test_size=0.01)

***In this cell the model is made and compiled.We use adam as optimizer and binary_crossentropy as loss***

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),#inputting our hyperparameters to create word embeddings 16 dimenstion
    tf.keras.layers.Bidirectional(LSTM(100,activation='relu')),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 64, 16)            1600000   
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               93600     
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                3216      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,697,105
Trainable params: 1,697,105
Non-trainable params: 0
______________________________________________

***This cell is used to save the model after every epoch although it will only save the model if the accuracy is better than the previous epoch***

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/email_classifier/models/best1_model.hdf5', monitor='accuracy', verbose=1,
save_best_only=True, mode='auto', period=1)

***In this cell we train the data and save models of it.You can train the model again to improve the accuracy.Just run the cell again***



---



In [ ]:
training=model.fit(X_train, Y_train, epochs=20, callbacks=[checkpoint], validation_data=(X_val, Y_val))

Epoch 1/20
160/160 [==============================] - ETA: 0s - loss: 0.4921 - accuracy: 0.8693
Epoch 00001: accuracy improved from -inf to 0.86926, saving model to /content/drive/My Drive/Colab Notebooks/email_classifier/models/best1_model.hdf5
160/160 [==============================] - 17s 109ms/step - loss: 0.4921 - accuracy: 0.8693 - val_loss: 7.6463 - val_accuracy: 0.9423
Epoch 2/20
160/160 [==============================] - ETA: 0s - loss: 29077812.0000 - accuracy: 0.9091
Epoch 00002: accuracy improved from 0.86926 to 0.90913, saving model to /content/drive/My Drive/Colab Notebooks/email_classifier/models/best1_model.hdf5
160/160 [==============================] - 18s 112ms/step - loss: 29077812.0000 - accuracy: 0.9091 - val_loss: 132425.5469 - val_accuracy: 0.8846
Epoch 3/20
160/160 [==============================] - ETA: 0s - loss: 15533.3203 - accuracy: 0.8403
Epoch 00003: accuracy did not improve from 0.90913
160/160 [==============================] - 23s 142ms/step - loss: 1

In [ ]:
acc = training.history['accuracy']
acc_list = (acc)
acc_max = max(acc_list)
print("the accuracy is ", acc_max)

the accuracy is  0.9980457425117493
